##🎯 What will you learn ?

- Understand what Sentinel-1 SAR data represents and why it works in all weather.
- Load and visualize Sentinel-1 (SAR) And Sentinel-2 (RGB) Image.

- Compare radar and optical data over the same agricultural area.
- Use a split map to visually explain SAR vs RGB differences.

In [ ]:
# Import Google Earth Engine and geemap
import ee
import geemap

In [ ]:
# Authenticate to Earth Engine.
ee.Authenticate()

# Initialize Earth Engine with your project ID
ee.Initialize(project =  'ee-nouribrahim25')

In [ ]:
# Define AOI
coors = [[29.194933, 29.859485],
    [29.194933, 31.765325],
    [32.656767, 31.765325],
    [32.656767, 29.859485],
    [29.194933, 29.859485]]

aoi = ee.Geometry.Polygon(coors)
aoi

In [ ]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.add_layer(aoi , {} , 'AOI')
Map.centerObject(aoi , 8)
Map

In [ ]:
# Date Range
start_date = '2025-01-01'
end_date = '2025-04-01'

###PART 1 — SENTINEL-1 (SAR)

Load Sentinel-1 GRD Collection

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD#colab-python

In [ ]:
# Define Function to Mask out noisy edges of Sentinel-1 images
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

In [ ]:
# Load S-1 Data
s1 = (ee.ImageCollection("COPERNICUS/S1_GRD")
      .filterBounds(aoi)
      .filterDate(start_date , end_date)
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
      .filter(ee.Filter.eq('instrumentMode' , 'IW'))  # Interferometric Wide swath
      .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))  # Use descending passes
      .select (['VV' , 'VH'])   # Select polarization bands
      .map(mask_edge)     #Apply edge masking
      .median()         #Create median composite
      .clip(aoi)
    )

s1

##PART 2 - Sentinel-2 (RGB)

**Load Sentinel-2 Data**

https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED#colab-python

Note :

We are not using cloud masking for the Sentinel-2 image.

This allows us to see the clouds in the RGB image and compare them with the SAR image, which can see right through them!

In [ ]:
# Load Sentinel-2
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(aoi)
    .filterDate(start_date, end_date)
    .select(['B4', 'B3', 'B2'])  # Red, Green, Blue bands
    .map(lambda image: image.divide(10000))  # Scale to reflectance (0-1)
    .median()
    .clip(aoi)
)

s2

##PART 3 - Create Split Map Visualization

**Define visualization parameters**

In [ ]:
# Sentinel-1: Radar values usually range from -25 (dark) to 5 (bright)
s1_vis = {
    'min': -25,     # Dark areas (low backscatter - water, smooth surfaces)
    'max': 5,       # Bright areas (high backscatter - buildings, rough surfaces)
    'bands': ['VH', 'VV', 'VV']     # We repeat 'VV' here to fill the Red, Green, and Blue color channels.
}

# Sentinel-2
s2_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],      # Red, Green , Blue bands
    'gamma': 1.4        # Enhance contrast
}

In [ ]:
# Create split panel map
left_layer = geemap.ee_tile_layer(s1, s1_vis, 'Sentinel-1')
right_layer = geemap.ee_tile_layer(s2, s2_vis, 'Sentinel-2 RGB')

Map.split_map(
    left_layer,
    right_layer,
    left_label="Sentinel-1 (SAR)",
    right_label="Sentinel-2 (RGB)",
)
Map